In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import print_function
import matplotlib.pyplot as plt

import autograd.numpy as np
import autograd.numpy.random as npr

from autograd import grad
from autograd.misc.optimizers import adam

import pandas as pd
from pandas import DataFrame

In [2]:
with open('../data/state_features.txt') as f:
    state_features = f.read().split()
print (state_features)
print (len(state_features))

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission', 'bloc']
48


In [3]:
df_train = pd.read_csv('../data/rl_train_set_unscaled.csv')
df_val = pd.read_csv('../data/rl_val_set_unscaled.csv')
df_test = pd.read_csv('../data/rl_test_set_unscaled.csv')

In [4]:
# let the mortality labels  be -1 and 1: -1 for survival
df_train['died_in_hosp'][df_train['died_in_hosp'] == 0] = -1
df_val['died_in_hosp'][df_val['died_in_hosp'] == 0] = -1
df_test['died_in_hosp'][df_test['died_in_hosp'] == 0] = -1

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
target_feat = list(np.loadtxt('../data/state_features_pred.txt', dtype=str))
target_feat.append('died_in_hosp')

cur_feat = list(np.loadtxt('../data/state_features.txt', dtype=str))
cur_feat.append('iv_input')
cur_feat.append('vaso_input')

In [6]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [7]:
inv_action_map = {}
for iv in range(5):
    for vaso in range(5):
        inv_action_map[5*iv+vaso] = [iv,vaso]    

In [8]:
# Change the actions: {0: -0.8, 1: -0.4, 2: 0, 3: 0.4, 4: 0.8}. This will help for the policy search, which
# uses a continuous action space
lookup ={0:-0.8, 1:-0.4, 2:0, 3:0.4, 4:0.8}
for dfm in (df_train, df_val, df_test):
    for iv_input in range(5):
        val = lookup[iv_input]
        dfm['iv_input'][dfm['iv_input'] == iv_input] = val
    for vaso_input in range(5):
        val = lookup[vaso_input]
        dfm['vaso_input'][dfm['vaso_input'] == vaso_input] = val

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [9]:
#  X: (states, actions)
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
def make_data(df_in):
    X = []
    Y = []
    for count,i in enumerate(df_in.index):
        cur_state = df_in.loc[i,cur_feat]
        if i != df_in.index[-1]:
            # if not terminal step in trajectory             
            if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
                target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
                target[-1] = df_in.loc[i, 'died_in_hosp']
                Y.append(target)
                X.append(cur_state)

        if count % 10000 == 0 and count > 0:
            print(count)

    return np.array(X),np.array(Y)

In [10]:
import os
dire = 'converted_data/'
if not os.path.exists(dire):
    os.makedirs(dire)
    
if not os.path.exists(dire + 'X_train.txt'):
    x_train_nohist, y_train_nohist = make_data(df_train)
    np.savetxt(dire + 'X_train.txt',x_train_nohist,fmt='%5.4f')
    np.savetxt(dire + 'Y_train.txt',y_train_nohist,fmt='%5.4f')
    print("Saved train")
else:
    x_train_nohist = np.loadtxt(dire + 'X_train.txt')
    y_train_nohist = np.loadtxt(dire + 'Y_train.txt')
    print ("Loaded train")

if not os.path.exists(dire + 'X_val.txt'):
    x_val_nohist,y_val_nohist = make_data(df_val)
    np.savetxt(dire + 'X_val.txt',x_val_nohist,fmt='%5.4f')
    np.savetxt(dire + 'Y_val.txt',y_val_nohist,fmt='%5.4f')
    print ("Saved val")
else:
    x_val_nohist = np.loadtxt(dire + 'X_val.txt')
    y_val_nohist = np.loadtxt(dire + 'Y_val.txt')
    print ("Loaded val")
    
if not os.path.exists(dire + 'X_test.txt'):
    x_test_nohist, y_test_nohist = make_data(df_test)
    np.savetxt(dire + 'X_test.txt',x_test_nohist,fmt='%5.4f')
    np.savetxt(dire +    'Y_test.txt',y_test_nohist,fmt='%5.4f')
    print ("Saved test")
else:
    x_test_nohist = np.loadtxt(dire + 'X_test.txt')
    y_test_nohist = np.loadtxt(dire + 'Y_test.txt')
    print ("Loaded test")

Loaded train
Loaded val
Loaded test


In [11]:
dire = 'converted_data/'
if not os.path.exists(dire):
    os.makedirs(dire)
    
if not os.path.exists(dire + 'X_train_hist.txt'):
    x_train_hist, y_train_hist = make_data_history(df_train)
    np.savetxt(dire + 'X_train_hist.txt',x_train_hist,fmt='%5.4f')
    np.savetxt(dire + 'Y_train_hist.txt',y_train_hist,fmt='%5.4f')
    print ("Saved train")
else:
    x_train_hist = np.loadtxt(dire + 'X_train_hist.txt')
    y_train_hist = np.loadtxt(dire + 'Y_train_hist.txt')
    print ("Loaded train")

if not os.path.exists(dire + 'X_val_hist.txt'):
    x_val_hist,y_val_hist = make_data_history(df_val)
    np.savetxt(dire + 'X_val_hist.txt',x_val_hist,fmt='%5.4f')
    np.savetxt(dire + 'Y_val_hist.txt',y_val_hist,fmt='%5.4f')
    print ("Saved val")
else:
    x_val_hist = np.loadtxt(dire + 'X_val_hist.txt')
    y_val_hist = np.loadtxt(dire + 'Y_val_hist.txt')
    print ("Loaded val")
    
if not os.path.exists(dire + 'X_test_hist.txt'):
    x_test_hist, y_test_hist = make_data_history(df_test)
    np.savetxt(dire + 'X_test_hist.txt',x_test_hist,fmt='%5.4f')
    np.savetxt(dire + 'Y_test_hist.txt',y_test_hist,fmt='%5.4f')
    print ("Saved test")
else:
    x_test_hist = np.loadtxt(dire + 'X_test_hist.txt')
    y_test_hist = np.loadtxt(dire + 'Y_test_hist.txt')
    print ("Loaded test")

Loaded train
Loaded val
Loaded test


In [12]:
x_train = x_train_hist
y_train = y_train_hist

x_val = x_val_hist
y_val = y_val_hist

x_test = x_test_hist
y_test = y_test_hist

In [13]:
x_train.shape

(119555, 200)

In [14]:
# get random sample of train/test for this model
train_idx = np.random.permutation(len(x_train))
val_idx = np.random.permutation(len(x_val))

x_train = x_train[train_idx]
y_train = y_train[train_idx]

x_val = x_val[val_idx]
y_val = y_val[val_idx]

In [15]:
# # only predict sofa
# y_train = np.expand_dims(y_train[:, 29], 1)
# y_val = np.expand_dims(y_val[:,29],1)

In [16]:
def black_box_variational_inference(logprob, D, num_samples):
    """Implements http://arxiv.org/abs/1401.0118, and uses the
    local reparameterization trick from http://arxiv.org/abs/1506.02557"""

    def unpack_params(params):
        # Variational dist is a diagonal Gaussian.
        mean, log_std = params[:D], params[D:]
        return mean, log_std

    def gaussian_entropy(log_std):
        return 0.5 * D * (1.0 + np.log(2*np.pi)) + np.sum(log_std)

    rs = npr.RandomState(0)
    def variational_objective(params, t):
        """Provides a stochastic estimate of the variational lower bound."""
        mean, log_std = unpack_params(params)
        samples = rs.randn(num_samples, D) * np.exp(log_std) + mean
        lower_bound = gaussian_entropy(log_std) + np.mean(logprob(samples, t))
        return -lower_bound

    gradient = grad(variational_objective)

    return variational_objective, gradient, unpack_params

In [17]:
def make_nn_funs(layer_sizes, L2_reg, noise_variance, nonlinearity=np.tanh):
    """These functions implement a standard multi-layer perceptron,
    vectorized over both training examples and weight samples."""
    shapes = list(zip(layer_sizes[:-1], layer_sizes[1:]))
    num_weights = sum((m+1)*n for m, n in shapes)

    def unpack_layers(weights):
        num_weight_sets = len(weights)
        for m, n in shapes:
            yield weights[:, :m*n]     .reshape((num_weight_sets, m, n)),\
                  weights[:, m*n:m*n+n].reshape((num_weight_sets, 1, n))
            weights = weights[:, (m+1)*n:]

    def predictions(weights, inputs):
        """weights is shape (num_weight_samples x num_weights)
           inputs  is shape (num_datapoints x D)"""
        inputs = np.expand_dims(inputs, 0)
        for W, b in unpack_layers(weights):
            outputs = np.einsum('mnd,mdo->mno', inputs, W) + b
            inputs = nonlinearity(outputs)
        return outputs

    def logprob(weights, inputs, targets):
        log_prior = -L2_reg * np.sum(weights**2, axis=1)
        preds = predictions(weights, inputs)
        log_lik = -np.sum((preds - targets)**2, axis=1)[:, 0] / noise_variance
        return log_prior + log_lik

    return num_weights, predictions, logprob

In [ ]:
# Specify inference problem by its unnormalized log-posterior.
from IPython import display

import pylab as pl

inputs, targets = (x_train, y_train)

rbf = lambda x: 1.5*np.exp(-x**2)
relu = lambda x: np.maximum(x, 0.01*x)
tanh = lambda x: 2*np.tanh(x)
num_weights, predictions, logprob = \
    make_nn_funs(layer_sizes=[x_train.shape[1], 32, 32, y_train.shape[1]], L2_reg=1.5,
                 noise_variance=0.01, nonlinearity=rbf)

batch_size = 10000

n_epoch = 5

num_batches = int(np.ceil(len(inputs) / batch_size))

glob_params = None

def batch_indices(iter):
    batchidx = iter % num_batches
    return slice(batchidx * batch_size, (batchidx+1) * batch_size)


def log_posterior(weights, t):
    cur_batch_idx = batch_indices(t)
    cur_inp = inputs[cur_batch_idx]
    cur_tar = targets[cur_batch_idx]
    return logprob(weights, cur_inp, cur_tar)

# Build variational objective.
objective, gradient, unpack_params = \
    black_box_variational_inference(log_posterior, num_weights,
                                    num_samples=20)


def callback(params, t, g):
    global glob_params
    print("Iteration {} lower bound {}".format(t, -objective(params, t)))
    
    glob_params = params

    # Sample functions from posterior.
    rs = npr.RandomState(0)
    mean, log_std = unpack_params(params)
    #rs = npr.RandomState(0)
    sample_weights = rs.randn(10, num_weights) * np.exp(log_std) + mean

    val_outputs = np.mean(predictions(sample_weights, x_val), axis=0)
#     print(val_outputs.shape)
#     import sys
#     sys.exit()
    
    mse = np.mean( (val_outputs - y_val)**2)
    
    print("Iteration {} val mse {}".format(t, mse))

#     # Plot data and functions.
#     if (t+1) % 10 == 0:
#         pl.plot(inputs.ravel(), targets.ravel(), 'bx')
#         pl.plot(plot_inputs, outputs[:, :, 0].T)
#     #     display.clear_output(wait=True)
#         pl.show()
#         print(np.squeeze(outputs).shape)

# Initialize variational parameters
rs = npr.RandomState(0)
init_mean    = rs.randn(num_weights)
init_log_std = -5 * np.ones(num_weights)
# init_mean, init_log_std = unpack_params(glob_params)
init_var_params = np.concatenate([init_mean, init_log_std])

print("Optimizing variational parameters...")
variational_params = adam(gradient, init_var_params,
                          step_size=0.05, num_iters=50 * num_batches, callback=callback)

Optimizing variational parameters...
Iteration 0 lower bound -8415512.43597
Iteration 0 val mse 16.2800800863
Iteration 1 lower bound -5657292.01343
Iteration 1 val mse 15.06208813
Iteration 2 lower bound -5446365.52002
Iteration 2 val mse 14.0220880141
Iteration 3 lower bound -4283670.81052
Iteration 3 val mse 12.8579109893
Iteration 4 lower bound -3470105.28335
Iteration 4 val mse 11.7164344725
Iteration 5 lower bound -3336549.19719
Iteration 5 val mse 10.7373255251
Iteration 6 lower bound -3441150.81132
Iteration 6 val mse 9.87003918964
Iteration 7 lower bound -3176898.3926
Iteration 7 val mse 9.06920879358
Iteration 8 lower bound -2685209.49385
Iteration 8 val mse 8.32821369825
Iteration 9 lower bound -2462161.11643
Iteration 9 val mse 7.6730106855
Iteration 10 lower bound -2416967.53738
Iteration 10 val mse 7.07703474228
Iteration 11 lower bound -2260372.31278
Iteration 11 val mse 6.5027117881
Iteration 12 lower bound -2015882.30246
Iteration 12 val mse 5.95548754902
Iteration 13 

In [56]:
np.save('glob_params.npy', glob_params)